- Utilisation de bigml pour récupérer le modèles d'apprentissage

In [65]:
from pandas import DataFrame, read_csv, Series
from bigml.api import BigML
api = BigML(project="project/5d94a42042129f2b0500005c")
model = api.get_ensemble("ensemble/5db162ae7811dd0557000e0e")

- Récupération des informations sur le candidat au prêt

In [66]:
raw_input = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}

- Mise en format pour faciliter la manipulation

In [67]:
df = DataFrame.from_dict([nouvel_input])
df['MonthlyIncome']=None
df['SeriousDlqin2yrs']=None
df['Unnamed: 0'] = None
df

,RevolvingUtilizationOfUnsecuredLines,NumberOfDependents,DebtRatio,age,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate,MonthlyIncome,SeriousDlqin2yrs
0,0.017036,1,0,42,6,1,1,0,0,None,None


- Modification des contenus vide (NA)

In [68]:
df['MonthlyIncome_sansna'] = df['MonthlyIncome'].fillna(0)

- Calcul de nouvelle features

In [69]:
df['Sum'] = df['NumberOfTime30-59DaysPastDueNotWorse'] + df['NumberOfTime60-89DaysPastDueNotWorse'] + df['NumberOfTimes90DaysLate']

In [70]:
df['IncomePerPerson'] = df['MonthlyIncome']/(df['NumberOfDependents']+1)

- Exportation des dataframe vers des fichiers csv pour envoi à bigml

In [71]:
df.to_csv('input_modified.csv')

- importation du fichier et création du dataset correspondant

In [72]:
source = api.create_source('input_modified.csv', {"name": "171107 input_modified"})
api.ok(source)
input_modified = api.create_dataset(source, {"name": "171107 input_modified"})
api.ok(input_modified)

True

- Evaluation du modèle sur le nouveau candidat

In [73]:
evaluation = api.create_evaluation(model, input_modified, {"name": "171107 input_modified - evaluation"})
api.ok(evaluation)

ValueError: {'code': -1, 'elapsed': 1532, 'error': -6070, 'message': 'No input row contains valid data', 'progress': 0}

- Affichage du résultat

In [ ]:
api.pprint(evaluation['object']['result']['model']['per_class_statistics'])